In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import gradio as gr
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
to

In [ ]:
# Example input text
sequence = """Machine learning is a subset of artificial intelligence that enables
computers to learn from data and make predictions or decisions without being explicitly programmed.
It involves algorithms that identify patterns in data, improve their accuracy over time,
and are widely used in various fields. In healthcare, machine learning helps detect diseases at
an early stage, while in finance, it assists in fraud detection and algorithmic trading.
E-commerce platforms use machine learning to personalize recommendations, and autonomous
vehicles rely on it for navigation and decision-making. However, challenges such as data bias,
model interpretability, and ethical concerns need to be addressed for responsible AI development."""

In [ ]:
sample='Quantum computing is an emerging field of computing that leverages the principles of quantum mechanics to perform computations at speeds unattainable by classical computers. Unlike traditional bits, which represent either 0 or 1, quantum bits (qubits) can exist in multiple states simultaneously due to superposition. Additionally, qubits can be entangled, meaning the state of one qubit is dependent on another, even across large distances. These properties enable quantum computers to solve complex problems in cryptography, drug discovery, and optimization far more efficiently than classical systems. However, challenges such as qubit stability, error correction, and hardware scalability remain obstacles to widespread adoption. Major tech companies and research institutions are investing heavily in quantum computing, with the hope of achieving quantum supremacy—the point at which a quantum computer outperforms the best classical computers for a specific task.'

In [ ]:
# Prepend "summarize: " for T5 input formatting
input_text = "summarize: " + sequence

In [ ]:
input_text = "summarize: " + sample

In [ ]:
# Tokenize and encode the input
inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

In [ ]:
def summarize(text):
    """Generate summary using T5 model."""
    input_text = "summarize: " + text
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

    summary_ids = model.generate(
        inputs,
        min_length=50, max_length=100,
        num_beams=5, length_penalty=1.5,
        no_repeat_ngram_size=3,
        early_stopping=True
    )

    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary


In [ ]:
iface = gr.Interface(
    fn=summarize,
    inputs=gr.Textbox(lines=10, placeholder="Enter full text here..."),
    outputs=gr.Textbox(lines=5, placeholder="Summary will appear here..."),
    title="T5 Text Summarization",
    description="Enter a long text and get a concise summary using a T5 model."
)

# Launch the app
iface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0dffdb4d8984ce2b32.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
from rouge_score import rouge_scorer

# Generate summary
output = model.generate(inputs, min_length=80, max_length=100)

# Decode and print the summary
summary = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Summary:\n", summary)

# Reference summary (gold standard)
reference_summary = 'Machine learning, a subset of artificial intelligence, enables computers to learn from data and make predictions without explicit programming. It identifies patterns, improves accuracy over time, and is widely applied across various fields. In healthcare, it aids in early disease detection, while in finance, it helps with fraud detection and trading. E-commerce platforms use it for personalized recommendations, and autonomous vehicles rely on it for navigation. However, challenges like data bias, model interpretability, and ethical concerns must be addressed for responsible AI development.'

# Function to calculate ROUGE scores
def calculate_rouge(reference_summary, generated_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)
    return scores

# Compute ROUGE scores
rouge_scores = calculate_rouge(reference_summary, summary)

# Display results
print("\nROUGE Scores:")
for metric, score in rouge_scores.items():
    print(f"{metric}: Precision={score.precision:.4f}, Recall={score.recall:.4f}, F1-Score={score.fmeasure:.4f}")



Generated Summary:
 machine learning is a subset of artificial intelligence that enables computers to learn from data . it involves algorithms that identify patterns in data and improve their accuracy over time . challenges such as data bias, model interpretability, and ethical concerns need to be addressed . but, challenges such as data bias, model interpretability, and ethical concerns need to be addressed . a machine learning expert says the future of artificial intelligence is uncertain .

ROUGE Scores:
rouge1: Precision=0.4857, Recall=0.4048, F1-Score=0.4416
rouge2: Precision=0.3043, Recall=0.2530, F1-Score=0.2763
rougeL: Precision=0.4429, Recall=0.3690, F1-Score=0.4026


In [ ]:
# Generate summary
output = model.generate(inputs, min_length=80, max_length=100)

# Decode and print the summary
summary = tokenizer.decode(output[0], skip_special_tokens=True)
print("Generated Summary:\n", summary)

Generated Summary:
 quantum computing is an emerging field of computing that leverages the principles of quantum mechanics . qubits can exist in multiple states simultaneously due to superposition, says robert mcdonald . challenges such as qubit stability, error correction, and hardware scalability remain obstacles to widespread adoption . mcdonald: major tech companies and research institutions are investing heavily in quantum computing .
